In [9]:
import openai
import pandas as pd
import os
import json
import numpy as np

전체 출력 설정
pd.set_option('display.max_colwidth', None)  # 한 셀에 출력될 최대 너비를 제한 없이 설정

In [16]:
# 현재 디렉토리에서 data파일을 찾은 후 xlsx파일 주소 합쳐서 데이터 불러오는 코드
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'data')
data_file_path = os.path.join(data_directory, '오픈마켓 노쇼핑 샘플 데이터.xlsx')
df = pd.read_excel(data_file_path)
# OpenAI API Key를 환경 변수에서 가져오기
openai.api_key = os.getenv("OPEN_API_KEY")

C:\Users\sera\AppData\Local\Temp\ipykernel_82468\34194009.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel(data_file_path)


In [17]:
def extract_fields(description):
    prompt = f"""
    우리는 여러 플랫폼에서 크롤링한 데이터를 통해 호텔과 숙박에 관한 노쇼 상품들을 판매할거야. 
    기본 필드에서 가져올 수 있는 정보들을 제외하고 우리가 필요한 필드가 10개가 있어. 
    만약 NaN이라면 그냥 모든 필드를 JSON의 `null`로 표시해줘.
    이 10개의 필드는 
    1. 노쇼 상품의 사용 날짜 및 기한을 나타내는 expiration_date
       이때 날짜가 범위로 표현되어 있다면 범위를 나타내고, 날짜가 없다면 JSON의 `null`로 표기해줘. 날짜 형식은 20xx-mm-dd로 통일해줘
    2. 객실 타입을 나타내는 room_type
       문자열 형태이며 관련된 내용이 없다면 JSON의 `null`로 입력해줘.
    3. 사용 인원 수를 나타내는 head_count
       데이터 타입은 int type이고 성인이나 자녀에 대한 구분 없이 총 인원수만 입력해줘. 인원수에 대한 내용이 없다면 JSON의 `null`로 기입해줘.
    4. 배송비인 shipping_fee 
       데이터 타입은 int type이고, 배송비가 적혀있지 않다면 0으로 표기해줘 (숫자 형식).
    5. 직거래인지 택배거래인지 알 수 있는 거래 유형인 transaction_method
       직거래라는 단어가 있으면 '직거래'로, 택배거래라는 단어가 있으면 '택배거래'로 표기해줘.
       두 단어가 모두 없으면 JSON의 `null`로 표기해줘.
    6. 원래 노쇼 상품의 가격인 시중 가격인 market_price 
       데이터 타입은 int type이고, 시중 가격이 나와 있지 않다면 JSON의 `null`로 표기해줘
       만약 23.3만원 이런식으로 적혀져 있다면 market_price는 233000으로 입력되어야해.
    7. 조식 여부나 사우나 여부, 수영장 여부 등의 호텔에서 사용할 수 있는 서비스 이용 여부를 알 수 있는 옵션인 options
       서비스 관련 내용이 없다면 JSON의 `null`로 표기해줘.
    8. 주차 여부를 나타내는 parking
       주차 여부가 있으면 TRUE로, 없으면 FALSE로 표기해줘. 주차 여부 정보가 없다면 JSON의 `null`로 표기해줘.
    9. 호텔 체크인 시간을 나타내는 check_in_time
       체크인 시간을 time 형태로 작성해줘. 만약 체크인 시간이 없으면 JSON의 `null`로 작성해줘.
    10. 호텔 체크아웃 시간을 나타내는 check_out_time
       체크아웃 시간을 time 형태로 작성해줘. 만약 체크아웃 시간이 없으면 JSON의 `null`로 작성해줘.
       
    설명에서 필요한 10개의 필드 정보만 추출해 json 형태로 작성해줘:
    예시: 

        "expiration_date": "2023-10-01",
        "room_type": "더블룸",
        "head_count": 2,
        ...
    
    설명: {description}

    필드:
    - expiration_date:
    - room_type:
    - head_count:
    - shipping_fee:
    - transaction_method:
    - market_price:
    - options:
    - parking:
    - check_in_time:
    - check_out_time:
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=1000
    )
    

    result = response['choices'][0]['message']['content'].strip()
    
    print("API 반환 결과:", result)
    
# json으로 시작되는 쓸데없는 부분을 제거 / 끝부분에 '들어가는 부분 제거 코드
    if result.startswith("```json"):
        result = result.replace("```json", "").strip()
    if result.endswith("```"):
        result = result.replace("```", "").strip()
        
   # 'null'을 실제 JSON의 null로 변환
    result = result.replace('"null"', 'null')

    try:
        return json.loads(result)
    except json.JSONDecodeError:
        print("유효하지 않은 JSON 형식입니다.")
        return None

In [14]:
# 1행은 그냥 빈 칸이라 뺌.
df = df.iloc[1:,:]

In [18]:
df_sample = df.sample(n=10)
json_to_df= df_sample['description'].apply(extract_fields)
# 데이터프레임으로 만드는 코드
fields_df = pd.json_normalize(json_to_df)
fields_df

API 반환 결과: {
    "expiration_date": null,
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": null,
    "market_price": null,
    "options": null,
    "parking": null,
    "check_in_time": null,
    "check_out_time": null
}
API 반환 결과: {
    "expiration_date": "2023-10-05",
    "room_type": "캡슐 프리미엄 더블(여성전용)",
    "head_count": 1,
    "shipping_fee": 0,
    "transaction_method": "택배거래",
    "market_price": 0,
    "options": null,
    "parking": null,
    "check_in_time": null,
    "check_out_time": null
}
API 반환 결과: {
    "expiration_date": "2022-11-15",
    "room_type": "패밀리룸",
    "head_count": 5,
    "shipping_fee": 0,
    "transaction_method": null,
    "market_price": 80000,
    "options": "조식 포함",
    "parking": null,
    "check_in_time": null,
    "check_out_time": null
}
API 반환 결과: {
    "expiration_date": null,
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": null,
    "market_price": nul

,expiration_date,room_type,head_count,shipping_fee,transaction_method,market_price,options,parking,check_in_time,check_out_time
0,None,None,NaN,0,None,NaN,None,None,None,None
1,2023-10-05,캡슐 프리미엄 더블(여성전용),1.0,0,택배거래,0.0,None,None,None,None
2,2022-11-15,패밀리룸,5.0,0,None,80000.0,조식 포함,None,None,None
3,None,None,NaN,0,None,NaN,None,None,None,None
4,2024-12-31,None,NaN,0,None,NaN,None,None,None,None
5,2023-11-01,None,NaN,0,None,NaN,None,None,None,None
6,None,None,NaN,0,None,NaN,None,None,None,None
7,None,패밀리 취사 스탠다드,6.0,0,None,NaN,"오션플레이 워터파크, 놀거리",True,None,None
8,None,더블룸,NaN,0,직거래,NaN,None,True,None,None
9,2021-10-06,None,0.0,0,None,NaN,None,None,None,None


In [19]:
fields_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   expiration_date     5 non-null      object 
 1   room_type           4 non-null      object 
 2   head_count          4 non-null      float64
 3   shipping_fee        10 non-null     int64  
 4   transaction_method  2 non-null      object 
 5   market_price        2 non-null      float64
 6   options             2 non-null      object 
 7   parking             2 non-null      object 
 8   check_in_time       0 non-null      object 
 9   check_out_time      0 non-null      object 
dtypes: float64(2), int64(1), object(7)
memory usage: 928.0+ bytes
